In [1]:
from random import randint
from string import Template
import json
from dataclasses import dataclass
import os
import shutil

from cog.torque import Graph
import pygraphviz as pgv
from IPython.display import Image

In [2]:
def all_subclasses(cls):
    return list(set(cls.__subclasses__()).union(
        [s for c in cls.__sublcasses__() for s in all_subclasses(c)]))

In [3]:
class QuantumSystem:
    quantum_system_name = "DMA"
    graph = None
    t = 0
    
    @classmethod
    def get_system(cls, name):
        for system in cls.MetaSystem.all_metasystems():
            if system.__name__ == name:
                return system
        
        for system in cls.HyperSystem.all_hypersystems():
            if system.__name__ == name:
                return system
        
        for system in cls.System.all_systems():
            if system.__name__ == name:
                return system

        raise Exception(f"Unknown system: {name}")
    
    @dataclass
    class Entity:
        """ An Entity is an anchor onto which Systems and HyperSystems attach """
        entity_id: str
        
        @classmethod
        def get(cls, entity_id):
            """Retrieve the Graph Node at `entity_id` and all immediate neighbors with edges"""
            return cls(entity_id, **{
                e: node['id']
                for node in QuantumSystem.v(entity_id).out().all('e')['result']
                for e in node['edges']})
    
    class System:
        """ Systems describe relationships between nouns """
        
        @classmethod
        def all_systems(cls):
            for subclass in cls.__subclasses__():
                yield from subclass.all_systems()
                yield subclass
        
        @classmethod
        def resolver(cls, entity_name, relationships):
            if cls.__name__ in relationships:
                QuantumSystem.put(entity_name, cls.__name__, relationships[cls.__name__])
                return cls.__name__
        
        @classmethod
        def relationships(cls):
            return QuantumSystem.v().tag(cls.__name__).inc(cls.__name__).all()['result']
        
        @classmethod
        def relationships_by_id(cls):
            return {rel['id']: rel for rel in cls.relationships()}
        
        @classmethod
        def get_edge_names(cls):
            return [cls.__name__]
        
        @classmethod
        def validate(cls, terminant):
            return terminant
        
        @classmethod
        def build_system(cls, anchor, terminant):
            validated = cls.validate(terminant)
            QuantumSystem.add_entity(anchor, **{cls.__name__: validated})
            return anchor
        
        @classmethod
        def get_relationship(cls, entity_id):
            try:
                return QuantumSystem.v(entity_id).out(cls.__name__).all()['result'][0]['id']
            except IndexError:
                return None
        
        @classmethod
        def set_relationship(cls, entity_id, terminant):
            existing = cls.get_relationship(entity_id)
            if existing is not None:
                QuantumSystem.drop(entity_id, cls, cls.validate(existing))
            cls.build_system(entity_id, terminant)

        @classmethod
        def decorator(cls, entity_id_name=None, returned_name=None):
            entity_id_name = entity_id_name or "entity_id"
            returned_name = returned_name or cls.__name__.lower()
        
            def decorator(func):
                def wrapped(*args, **kwargs):
                    if cls.__name__.lower() not in kwargs:
                        entity_id = kwargs[entity_id_name]
                        kwargs[returned_name] = cls.get_relationship(entity_id)
                    return func(*args, **kwargs)
                return wrapped
            return decorator

        @classmethod
        def d(cls, func):
            return cls.decorator()(func)

        @classmethod
        def system_decorator(cls, system):
            setattr(system, cls.__name__, cls)
            system.systems.append(cls)
            return system

        @classmethod
        def sd(cls, system):
            return cls.system_decorator(system)
        
        
    class HyperSystem:
        """ HyperSystems describe relationships between relationships of nouns """
        systems = []
        
        @classmethod
        def all_hypersystems(cls):
            for subclass in cls.__subclasses__():
                yield from subclass.all_hypersystems()
                yield subclass
        
        @classmethod
        def get_edge_names(cls):
            return [system.get_edge_name() for system in cls.systems]
        
        @classmethod
        def validate(cls):
            [system.validate() for system in cls.systems]
        
        @classmethod
        def build_system(cls, anchor, terminants: list | dict):
            # print(f"Building system: {cls} on {anchor} with {terminants}")
            if isinstance(terminants, dict):
                try:
                    terminants = [(sys, terminants[sys.__name__]) for sys in cls.systems]
                except KeyError:
                    raise Exception(f"Missing terminant for {sys.__name__}")
            else:
                terminants = list(zip(cls.systems, terminants))
                if len(terminants) != len(cls.systems):
                    raise Exception(f"Missing terminants for {[sys.__name__ for sys in cls.systems[len(terminants):]]}")
            for system, terminant in terminants:
                if terminant is not None:
                    system.build_system(anchor, terminant)
            return anchor

        @classmethod
        def system_decorator(cls, system):
            setattr(system, cls.__name__, cls)
            system.systems.append(cls)
            return system

        @classmethod
        def sd(cls, system):
            return cls.system_decorator(system)
    
    class MetaSystem:
        """ MetaSystems encapsulate HyperSystems and Systems """
        systems = []
        
        @classmethod
        def all_metasystems(cls):
            for subclass in cls.__subclasses__():
                yield from subclass.all_metasystems()
                yield subclass
        
        @classmethod
        def get_edge_names(cls):
            return [system.get_edge_name() for system in cls.systems]

        @classmethod
        def meta_anchor(cls, anchor):
            return f"{anchor}-{cls.__name__}"  
        
        @classmethod
        def build_system(cls, anchor, terminants):
            meta_anchor = cls.meta_anchor(anchor)
            if isinstance(terminants, dict):
                try:
                    terminants = [(sys, terminants[sys.__name__]) for sys in cls.systems]
                except KeyError:
                    raise Exception(f"Missing terminant for {sys.__name__}")
            else:
                terminants = list(zip(cls.systems, terminants))
                if len(terminants) != len(cls.systems):
                    raise Exception(f"Missing terminants for {[sys.__name__ for sys in cls.systems[len(terminants):]]}")
            
            for system, terminant in terminants:
                system.build_system(meta_anchor, terminant)
            QuantumSystem.add_entity(anchor, **{cls.__name__: meta_anchor})
            return meta_anchor

        @classmethod
        def system_decorator(cls, system):
            setattr(system, cls.__name__, cls)
            system.systems.append(cls)
            return system

        @classmethod
        def sd(cls, system):
            return cls.system_decorator(system)

    @classmethod
    def reset(cls):
        if os.path.exists('/tmp/cog_home/'):
            shutil.rmtree('/tmp/cog_home')
        cls.graph = Graph(cls.quantum_system_name)
    
    @classmethod
    def whole_graph(cls):
        return cls.graph.v().tag("from").out().tag("to")
    
    @classmethod
    def add_entity(cls, entity_id, **relationships):
        for rel_name, rel_value in relationships.items():
            try:
                cls.graph.put(entity_id, rel_name, rel_value)
            except Exception as e:
                print(f"Failed to create entity: {entity_id}, {rel_name}, {rel_value}")
                print(f"Exception: {e}")
                print(type(entity_id))
                print(type(rel_name))
                print(type(rel_value))
                raise
    
    @classmethod
    def entity_exists(cls, entity_id):
        return (cls.v(entity_id).inc().count() + cls.v(entity_id).out().count()) > 0
    
    @classmethod
    def v(cls, *args, **kwargs):
        return cls.graph.v(*args, **kwargs)
    
    @classmethod
    def put(cls, entity_id, edge_name, system_value):
        if type(edge_name) is type:
            edge_name = edge_name.__name__
        cls.graph.put(entity_id, edge_name, system_value)
    
    @classmethod
    def drop(cls, entity_id, edge_name, system_value):
        if type(edge_name) is type:
            edge_name = edge_name.__name__
        cls.graph.drop(entity_id, edge_name, system_value)
    
    @classmethod
    def replace(cls, entity_id, edge_name, old_value, new_value=None):
        cls.drop(entity_id, edge_name, old_value)
        if new_value is not None:
            cls.put(entity_id, edge_name, new_value)
    
    @classmethod
    def validate(cls):
        results = {}
        for system in list(cls.System.all_systems()) + list(cls.HyperSystem.all_hypersystems()):
            try:
                system.validate()
            except Exception as exc:
                    results[system.__name__] = exc
        return results
    
    @classmethod
    def trace_edges(cls, edge_name, g=None):
        if type(edge_name) is type:
            edge_name = edge_name.__name__
        g = cls.v("output") if g is None else g
        edges = g.inc(edge_name).all()['result']
        return [f['id'] for f in edges] + (trace_edges(g) if len(edges) > 0 else [])
    
    @staticmethod
    def path_dir(graph, reverse=True):
        return graph.inc if reverse else graph.out
    
    @classmethod
    def build_traversal_at_depth(cls, edge_name, depth, start=None, reverse=True):
        if type(edge_name) is type:
            edge_name = edge_name.__name__
        g = cls.v() if start is None else cls.v(start)
        for i in range(depth):
            g = cls.path_dir(g.tag(f"depth{i}"), reverse)(edge_name).tag("end")
        return g
    
    @classmethod
    def get_paths(cls, edge_name, end=None, start=None, reverse=True):
        if type(edge_name) is type:
            edge_name = edge_name.__name__
        g = cls.v(start) if start is not None else cls.v()
        depth = 0
        all_paths = []
        while (g := cls.build_traversal_at_depth(edge_name, depth, start=start, reverse=reverse)).count() > 0:
            for path in g.all()['result']:
                if cls.path_dir(cls.v(path['id']), reverse)(edge_name).count() == 0:
                    all_paths.append(path)
            depth += 1
        return all_paths
    
    @classmethod
    def render(cls):
        if pgv is None:
            raise Exception("Missing graphviz requirement")
        
        def color_selector(value, color_set='node'):
            edge_set = ["black", "royalblue1", "slateblue", "coral"]
            node_set = ["red", "webmaroon", "green", "chartreuse4", "aquamarine2"]
            color_set = node_set if color_set == 'node' else edge_set
            try:
                float(value)
                return "purple"
            except ValueError:
                pass
            
            if value not in color_selector.color_map:
                color_selector.color_map[value] = color_set[randint(0, len(color_set) - 1)]

            return color_selector.color_map[value]
        
        color_selector.color_map = {}
        
        vg = pgv.AGraph()
        nodes = []
        for rel in cls.whole_graph().all('e')['result']:
            if rel['to'] not in nodes:
                vg.add_node(rel['to'], color=color_selector(rel['to']))
                nodes.append(rel['to'])
            if rel['from'] not in nodes:
                vg.add_node(rel['from'], color=color_selector(rel['from']))
                nodes.append(rel['from'])
            vg.add_edge(rel['to'], rel['from'], label=rel['edges'][0],
                        color=color_selector(rel['edges'][0], color_set="edge"),
                        len=2)
        
        vg.layout(prog='neato')
        vg.draw(f'{cls.quantum_system_name}.png')
        return Image(f'{cls.quantum_system_name}.png')
qs = QuantumSystem